# Кузнецов Михаил Пи19-4

# Номер билета 34


# Задание 2



# Антагонистические игры



In [2]:
import pandas as pd
import numpy as np
import fractions

## Численный метод Брауна-Робинсон

In [3]:
class Game:
    def __init__(self, game_matrix: np.array, mixed_1: np.array, mixed_2: np.array, mixed_cost_1: float, mixed_cost_2: float):
        self.game_matrix = game_matrix
        self.mixed_1 = mixed_1
        self.mixed_2 = mixed_2
        self.mixed_cost_1 = mixed_cost_1
        self.mixed_cost_2 = mixed_cost_2

        self.pure_1 = 0
        self.pure_cost_1 = - np.inf
        n, m = game_matrix.shape
        for i in range(n):
            tmp = np.min(game_matrix[i, : ])
            if tmp > self.pure_cost_1:
                self.pure_1 = i
                self.pure_cost_1 = tmp
 
        self.pure_2 = 0
        self.pure_cost_2 = np.inf
        
        for i in range(m):
            tmp = np.max(game_matrix[ : ,i ])
            if tmp < self.pure_cost_2:
                self.pure_2 = i
                self.pure_cost_2 = tmp

    def __str__(self):
        return "".join([
        "mixed strat of А: {}\n".format(self.mixed_1),
        "mixed strat of Б: {}\n".format(self.mixed_2),
        "pure strat of А: {}\n".format(self.pure_1),
        "pure strat of Б: {}\n".format(self.pure_2),
        "mixed cost of А: {}\n".format(self.mixed_cost_1),
        "mixed cost of Б: {}\n".format(self.mixed_cost_2),
        "pure cost of А: {}\n".format(self.pure_cost_1),
        "pure cost of Б: {}\n".format(self.pure_cost_2)],
        )

class BrownRobinsonMethod:
    def __init__(self, eps = 10**-3):
        self.eps = eps

    def solve(self, game_matrix: np.array):
        m, n = game_matrix.shape

        # Вектора смешанных стратегий
        x = np.zeros(m)
        y = np.zeros(n)

        # Вектора выигрыша 1 игрока и проигрыша 2 игрока
        win_1 = np.zeros(m)
        loss_2 = np.zeros(n)

        # Текущие стратегии 1 и 2 игроков; в начале можно и случайный выбор
        cur_st_1 = 0 # np.random.randint(m)
        cur_st_2 = 0 # np.random.randint(n)

        lower_bounds = []
        upper_bounds = []


        step = 0
        cur_eps = np.inf
        while cur_eps > self.eps:
            # Каждый из игроков исходит от выбора другого игрока на шаге
            # Игрок 1 среди выбора стратегии игрока 2 пытается найти наиболее выигрышную стратегию
            # Игрок 2 среди выбора стратегии игрока 1 пытается найти наименее проигрышную стратегию
            step += 1

            win_1 = np.add(win_1, game_matrix[ : ,cur_st_2])
            loss_2 = np.add(loss_2, game_matrix[cur_st_1, : ])

            x[cur_st_1] += 1
            y[cur_st_2] += 1

            lower_bound = np.divide(np.min(loss_2), step)
            upper_bound = np.divide(np.max(win_1), np.double(step))
  
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)

            cur_eps = np.min(upper_bounds) - np.max(lower_bounds)

            cur_st_1 = np.argmax(win_1)
            cur_st_2 = np.argmin(loss_2)


        cost_1 = max(lower_bounds) + np.divide(cur_eps, 2)
        cost_2 = min(upper_bounds) - np.divide(cur_eps, 2)

        # Собственно, считаем, с какой вероятностью выбираем каждую стратегию
        x = np.divide(x, step)
        y = np.divide(y, step)

        # Чистая - наиболее вероятная в векторе смешанных
        return Game(game_matrix, x, y, cost_1, cost_2)

## Тест

In [4]:
example = np.array([
  [7, -6, 11, 9],
  [5, 8, 9, 5],
  [1, 7, 8, 2],
  [5, -6, 6, 7]
])

brm = BrownRobinsonMethod(10**-1)
sol = brm.solve(example)

strats_a = ["A1", "A2", "A3", "A4"]
strats_b = ["B1", "B2", "B3", "B4"]

print(sol)

mixed strat of А: [0.03076923 0.96923077 0.         0.        ]
mixed strat of Б: [0.84615385 0.15384615 0.         0.        ]
pure strat of А: 1
pure strat of Б: 0
mixed cost of А: 5.4125874125874125
mixed cost of Б: 5.4125874125874125
pure cost of А: 5
pure cost of Б: 7



### Смешанные стратегии игроков

In [6]:
df1 = pd.DataFrame(data = [sol.mixed_1], columns = strats_a)
df1

,A1,A2,A3,A4
0,0.030769,0.969231,0.0,0.0


In [7]:
df2 = pd.DataFrame(data = [sol.mixed_2], columns = strats_b)
df2

,B1,B2,B3,B4
0,0.846154,0.153846,0.0,0.0


#### Цена игры игроков A, B при выборе смешанной стратегии

In [8]:
print("Цена игры А при выборе смешанной стратегии: {}".format(sol.mixed_cost_1))
print("Цена игры Б при выборе смешанной стратегии: {}".format(sol.mixed_cost_2))

Цена игры А при выборе смешанной стратегии: 5.4125874125874125
Цена игры Б при выборе смешанной стратегии: 5.4125874125874125


### Чистые оптимальные стратегии игроков

In [9]:
print("Чистая стратегия игрока А: {}, стоимость игры: {}".format(sol.pure_1, sol.pure_cost_1))
print("Чистая стратегия игрока Б: {}, стоимость игры: {}".format(sol.pure_2, sol.pure_cost_2))

Чистая стратегия игрока А: 1, стоимость игры: 5
Чистая стратегия игрока Б: 0, стоимость игры: 7


#### Класс для более наглядного вывода

In [10]:
class GameData:
    def __init__(self):
        self.n = 0
        self.m = 0
        self.strats_a = None
        self.strats_b = None
        self.matrix = None
        self.solution = None

    def play(self):
        brm = BrownRobinsonMethod(10**-1)
        self.solution = brm.solve(self.matrix)

    def print_results(self):
        print("\nРезультаты решения:\n")
        print("Чистая стратегия игрока А: {} - {}, стоимость игры: {}".format(self.solution.pure_1 + 1, self.strats_a[self.solution.pure_1], self.solution.pure_cost_1))
        print("Чистая стратегия игрока Б: {} - {}, стоимость игры: {}".format(self.solution.pure_2 + 1, self.strats_b[self.solution.pure_2], self.solution.pure_cost_2))

        df1 = pd.DataFrame(data = [self.solution.mixed_1], columns = self.strats_a)
        print("Смешанная стратегия игрока А:\n{}".format(df1))
        print("Цена игры А при выборе смешанной стратегии: {}".format(self.solution.mixed_cost_1))

        df2 = pd.DataFrame(data = [self.solution.mixed_2], columns = self.strats_b)
        print("Смешанная стратегия игрока Б:\n{}".format(df2))
        print("Цена игры Б при выборе смешанной стратегии: {}".format(self.solution.mixed_cost_2))



#### Вывод

In [13]:
game = GameData()
game.matrix = example
game.strats_a = strats_a
game.strats_b = strats_b
print("Исходные данные: ")
print(game.matrix)
print(game.strats_a)
print(game.strats_b)
game.play()
game.print_results()

Исходные данные: 
[[ 7 -6 11  9]
 [ 5  8  9  5]
 [ 1  7  8  2]
 [ 5 -6  6  7]]
['A1', 'A2', 'A3', 'A4']
['B1', 'B2', 'B3', 'B4']

Результаты решения:

Чистая стратегия игрока А: 2 - A2, стоимость игры: 5
Чистая стратегия игрока Б: 1 - B1, стоимость игры: 7
Смешанная стратегия игрока А:
         A1        A2   A3   A4
0  0.030769  0.969231  0.0  0.0
Цена игры А при выборе смешанной стратегии: 5.4125874125874125
Смешанная стратегия игрока Б:
         B1        B2   B3   B4
0  0.846154  0.153846  0.0  0.0
Цена игры Б при выборе смешанной стратегии: 5.4125874125874125
